In [1]:
from z3 import *
import z3
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from itertools import islice

In [2]:
def parse_queries_from_klee_smt2_dump(path: str):
    queries = []
    with open(path) as f:
        current = ""
        for line in f:
            if line == "(check-sat)":
                continue
            if line.strip() == "(reset)":
                queries.append(z3.parse_smt2_string(current))
                current = ""
            current += line
    return queries        

In [4]:
queries = parse_queries_from_klee_smt2_dump("echo-cache.smt2")
print(f"{len(queries)} queries were loaded.")

4302 queries were loaded.


In [3]:
def calc_distance_keeping_constraint_order(first: list, second: list, pop_cost, push_cost):
    """
    Calculates a distance between two queries while not trying to change the order
    of constraints inside them.
    The returned distance mimics the cost of pushing and popping when the second query
    is solved right after the first query.
    """
    i = 0
    while i < len(first) and i < len(second) and first[i].eq(second[i]):
        i += 1

    return (i, pop_cost(len(first) - i) + push_cost(len(second) - i))
    # If you believe that there's no cost in popping, you can use the following
    # formula. But personally, I see each pop as a negative thing and losing a
    # progress.
    # return (i, len(second) - i)

def calc_distances_for(query_index: int, queries, calc_distance, pop_cost, push_cost):
    query = queries[query_index]
    distances = [0] * len(queries)
    for j in range(len(queries)):
        distances[j] = calc_distance(query, queries[j], pop_cost, push_cost)
    return distances

pop_cost = lambda x: x
push_cost = lambda x: x

def calc_distances_for_map(query_index: int):
    return calc_distances_for(query_index, queries, calc_distance_keeping_constraint_order, pop_cost, push_cost)

In [5]:
with ProcessPoolExecutor() as executor:
    distances = list(tqdm(executor.map(calc_distances_for_map, range(len(queries))), total=len(queries)))

common_prefix_lens = [[col[0] for col in row] for row in distances]
distances = [[col[1] for col in row] for row in distances]

100%|██████████| 4302/4302 [00:26<00:00, 165.37it/s]


In [6]:
def get_tsp_circuit(distances):
    import ortools.constraint_solver.pywrapcp as pywrapcp
    from ortools.constraint_solver import routing_enums_pb2

    manager = pywrapcp.RoutingIndexManager(len(distances), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distances[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

    def print_solution(manager, routing, solution):
        """Prints solution on console."""
        print('Objective: {} miles'.format(solution.ObjectiveValue()))
        index = routing.Start(0)
        plan_output = 'Route for vehicle 0:\n'
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        print(plan_output)
        plan_output += 'Route distance: {}miles\n'.format(route_distance)

    def get_vector(manager, routing, solution):
        index = routing.Start(0)
        route = []
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))

        return route

    solution = routing.SolveWithParameters(search_parameters)
    # if solution:
    #     print_solution(manager, routing, solution)

    return get_vector(manager, routing, solution)

In [7]:
best_case = get_tsp_circuit(distances)

In [8]:
from typing import List


def check_by_resetting(queries, ordering: List[int], common_prefix_lens, enable_direct_subset_answer=False, solver=None):
    solver = solver if solver is not None else Solver()
    results = {}
    
    total_time = 0.0
    
    for index in tqdm(ordering):
        query = queries[index]
        start_time = time.perf_counter()
        result = solver.check(query)
        end_time = time.perf_counter()
        total_time += end_time - start_time
        results[index] = result
    
    return (results, solver.statistics(), total_time, (0, 0, 0))

def check_incrementally(queries, ordering: List[int], common_prefix_lens, enable_direct_subset_answer=False, solver=None):
    solver = solver if solver is not None else Solver()
    results = {}

    queries = queries + [queries[ordering[-1]]]
    ordering = ordering + [ordering[-1]]
    
    last_index = len(queries) - 1
    current_stack_count = 0

    total_time = 0.0
    total_pops = 0
    total_pushes = 0

    for index, next_index in tqdm(zip(ordering[:len(ordering) - 1], ordering[1:]), total=len(queries) - 1):
        if index in results:
            print("Warning: Skipping repeated query. Query index =", index)
            continue
        # last_query = queries[last_index]
        query = queries[index]
        
        next_prefix_len = common_prefix_lens[index][next_index]

        start_time = time.perf_counter()
        if enable_direct_subset_answer and current_stack_count == len(query) and results[last_index] == sat:
            results[index] = sat
        else:
            if current_stack_count < next_prefix_len:
                for i in range(current_stack_count, next_prefix_len):
                    solver.push()
                    solver.add(query[i])
                total_pushes += next_prefix_len - current_stack_count
                current_stack_count = next_prefix_len

            solver.push()
            total_pushes += 1
            solver.add(query[current_stack_count:])
            
            result = solver.check()

            solver.pop()
            total_pops +=1
                        
            results[index] = result
        
        if current_stack_count > next_prefix_len:
            solver.pop(current_stack_count - next_prefix_len)
            total_pops += current_stack_count - next_prefix_len
            current_stack_count = next_prefix_len

        end_time = time.perf_counter()
        total_time += end_time - start_time
        last_index = index
    
    return (results, solver.statistics(), total_time, (total_pushes, total_pops, sum(len(q) for q in queries)))

In [9]:
# Base case
base_results, statistics, total_time, _ = check_by_resetting(queries, list(range(len(queries))), common_prefix_lens)
print("Spent time:", total_time)

100%|██████████| 4302/4302 [00:13<00:00, 318.86it/s] 

Spent time: 13.449079642188735


In [25]:
best_results, statistics, total_time, (total_pushes, total_pops, total_constraints) = check_incrementally(queries, list(range(len(queries))), common_prefix_lens)
assert(base_results == best_results)
print(total_time)
print(total_pops, total_pushes, total_constraints)

100%|██████████| 4302/4302 [00:09<00:00, 471.79it/s] 

9.07631946366746
149101 149108 679928


In [26]:
# Best case
best_results, statistics, total_time, (total_pushes, total_pops, total_constraints) = check_incrementally(queries, best_case, common_prefix_lens, True)
assert(base_results == best_results)
print(total_time)
print(total_pops, total_pushes, total_constraints)

100%|██████████| 4302/4302 [00:08<00:00, 492.59it/s] 

8.698012300301343
148177 148458 680202


In [10]:

def calc_distances():
    with ProcessPoolExecutor() as executor:
        distances = list(tqdm(executor.map(calc_distances_for_map, range(len(queries))), total=len(queries)))

    common_prefix_lens = [[col[0] for col in row] for row in distances]
    distances = [[col[1] for col in row] for row in distances]
    return common_prefix_lens, distances

def evaluate(name, enable_direct_subset_answer, output_suffix, repeat_count = 5, input_suffix = ".smt2"):
    # Using global because of pickling
    global queries
    queries = parse_queries_from_klee_smt2_dump(name + input_suffix)
    print(f"Evaluating {name} with {len(queries)} queries.")

    print("Calculating distances ...")
    common_prefix_lens, distances = calc_distances()
    
    original_ordering = list(range(len(queries)))
    print("Finding TSP ordering ...")
    tsp_ordering = get_tsp_circuit(distances)
    
    cases = {
        "res_original": (check_by_resetting, original_ordering),
        "incr_original": (check_incrementally, original_ordering),
        "incr_tsp": (check_incrementally, tsp_ordering)
    }

    results = {}
    check_results = []
    for (key, (check, ordering)) in cases.items():
        print(f"Testing {key} for {repeat_count} times ...")

        times = []
        for i in range(repeat_count):
            check_result, statistics, total_time, (total_pushes, total_pops, total_constraints) = check(queries, ordering, common_prefix_lens, enable_direct_subset_answer)
            times.append(total_time)
        check_results.append(check_result)
        print(total_time)
        
        results[key] = {
            "times": times,
            "statistics": str(statistics),
            "check_result": {key: str(value) for key, value in check_results[-1].items()},
            "ordering": ordering,
            "total_pushes": total_pushes,
            "total_pops": total_pops,
            "total_constraints": total_constraints,
        }
    
    assert all(result == check_results[0] for result in check_results)
    
    import json
    file_name = f"{name}_{output_suffix}.json"
    with open(file_name, "a") as f:
        json.dump(results, f)

    print("Result saved to:", file_name)

def zero_cost(x):
    return 0

def identity_cost(x):
    return x

push_cost = identity_cost
pop_cost = zero_cost
evaluate("echo-cache", True, "direct")

Evaluating echo-cache with 4302 queries.
Calculating distances ...
Finding TSP ordering ...
Testing res_original for 1 times ...


100%|██████████| 4302/4302 [00:13<00:00, 317.44it/s] 


13.509025148581713
Testing incr_original for 1 times ...


100%|██████████| 4302/4302 [00:08<00:00, 502.88it/s] 


8.522293226444162
Testing incr_tsp for 1 times ...


100%|██████████| 4302/4302 [00:08<00:00, 524.37it/s] 

8.179050556849688
[{0: sat, 1: sat, 2: sat, 3: sat, 4: sat, 5: sat, 6: unsat, 7: sat, 8: sat, 9: sat, 10: sat, 11: sat, 12: sat, 13: sat, 14: sat, 15: sat, 16: sat, 17: sat, 18: sat, 19: sat, 20: sat, 21: sat, 22: sat, 23: sat, 24: sat, 25: sat, 26: sat, 27: sat, 28: sat, 29: sat, 30: sat, 31: sat, 32: sat, 33: sat, 34: sat, 35: sat, 36: sat, 37: sat, 38: sat, 39: sat, 40: sat, 41: sat, 42: unsat, 43: sat, 44: sat, 45: sat, 46: sat, 47: unsat, 48: unsat, 49: sat, 50: sat, 51: sat, 52: sat, 53: unsat, 54: sat, 55: sat, 56: sat, 57: sat, 58: sat, 59: sat, 60: unsat, 61: sat, 62: unsat, 63: sat, 64: sat, 65: sat, 66: sat, 67: sat, 68: sat, 69: sat, 70: sat, 71: sat, 72: unsat, 73: unsat, 74: unsat, 75: sat, 76: sat, 77: sat, 78: sat, 79: sat, 80: unsat, 81: sat, 82: unsat, 83: sat, 84: sat, 85: sat, 86: sat, 87: sat, 88: sat, 89: sat, 90: sat, 91: sat, 92: sat, 93: unsat, 94: sat, 95: sat, 96: sat, 97: sat, 98: sat, 99: sat, 100: sat, 101: sat, 102: sat, 103: sat, 104: sat, 105: sat, 106: